In [1]:
import numpy as np
import pandas as pd
import joblib
import re
from collections import Counter
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)

df = pd.read_excel('data/Loyalty_dataset_v4.xlsx', 'Última Poliza', engine='openpyxl')

In [2]:
tipo_poliza_map = {
    'BIC': 'OFF Shore',
    'BINS': 'OFF Shore',
}

df['tipo_poliza'] = df.CoverEntity.map(tipo_poliza_map)
df['tipo_poliza'].fillna('ON Shore', inplace=True)

In [3]:
df.fillna(0, inplace=True)
df['max_age'] = df[['OwnerAge', 'SpouseAge'] + ['Unknown'+str(i) for i in range(1,10)]].max(axis=1)
df

,PH_ContactBaseID,CoverageFromDate,CoverageThruDate,PolicyYear,Orden,PolicyMonth,BillingFreq,PlanKey,RegionKey,CoverEntity,CoverProduct,CoverPlan,ProductGroup,RiskCountry,RateClassL0,RateClassL1,Duration,Members,FamilyStructure,GuarianAge,OwnerAge,SpouseAge,DependentsCountLT23,DependentsCountAT23,UnknownCount,Unknown1,Unknown2,Unknown3,Unknown4,Unknown5,Unknown6,Unknown7,Unknown8,Unknown9,Renewal,LapseMid,LapseFull,PremiumCalculator,PremiumIn,PremiumOut,PremiumOutConstant,RenewedAT23,flagAlcoholMisuse,flagAsthma,flagAtrialFibrillation,flagCancerLymphoma,flagCancerMetastatic,flagCancerNonMetastatic,flagChronicHeartFailure,flagChronicKidneyDisease,flagChronicPain,flagChronicPulmonaryDisease,flagChronicViralHepatitis,flagCirrhosis,flagDementia,flagDepression,flagDiabetes,flagEpilepsy,flagHypertension,flagHypothyroidism,flagInflammatoryBowel,flagIrritableBowel,flagMultipleSclerosis,flagMyocardialInfarction,flagParkinsons,flagPepticUlcer,flagPeripheralVascular,flagPsoriasis,flagRheumatoidArthritis,flagSchizophrenia,flagSevereConstipation,flagStroke,RateIncrease,RateIncreasePortfolio,RateIncreaseAge,PaidClaims,IncurredClaims,CountClaims,DenialsL12,DeadEndFlag,TransferBupaMexico,TransferBupaEcuador,TransferBupaDR,TransferBupaBolivia,TransferBupaPanama,TransferBupaGuatemala,SuccessorsFoundDiffNumber,tipo_poliza,max_age
0,251795,2020-02-01,2021-01-31,2020,1,2,Annual,AMP_E4V,AMP_Region Venezuela,BIC,Amedex Worldwide,Plan E,Mid,Venezuela,Amedex Worldwide,Venezuela,20,2,Married,0.0,62,58.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,Normal,9386,10051,10051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.070850,0.070850,0.000000,0.000,0.000000,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,62.0
1,251797,2019-02-01,2020-01-31,2019,1,2,Annual,AMP_E4V,AMP_Region Venezuela,BIC,Amedex Worldwide,Plan E,Mid,Venezuela,Amedex Worldwide,Venezuela,19,1,Single,0.0,32,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,Normal,1871,2121,2121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.133618,0.133618,0.000000,0.000,0.000000,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,32.0
2,251798,2019-02-01,2020-01-31,2019,1,2,Annual,AMP_E4V,AMP_Region Venezuela,BIC,Amedex Worldwide,Plan E,Mid,Venezuela,Amedex Worldwide,Venezuela,19,1,Single,0.0,29,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,Normal,1651,2121,1873,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.284676,0.134464,0.132408,0.000,0.000000,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,29.0
3,251806,2020-01-15,2021-01-14,2020,1,1,Semi-Annual,AMP_ET63,AMP_Region 3 Bupa,BIC,Bupa Care Essential,Plan 6,Low,El Salvador,Bupa Care Offshore,Region 3,20,2,Married,0.0,57,57.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,Normal,8158,9232,9232,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.131650,0.131650,0.000000,4199.998,4285.712245,29,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,57.0
4,251809,2020-01-15,2021-01-14,2020,1,1,Semi-Annual,AMP_ET63,AMP_Region 3 Bupa,BIC,Bupa Care Essential,Plan 6,Low,El Salvador,Bupa Care Offshore,Region 3,20,1,Single,0.0,25,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,Normal,1837,2083,2083,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.133914,0.133914,0.000000,0.000,0.000000,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59577,1284554,2020-02-01,2021-01-31,2020,1,2,Quarterly,AMP_AW41,AMP_Region 1 Bupa,BIC,Bupa Care Advantage,Plan 4,Mid,Mexico,Bupa Care Offshore,Region 1,12,1,Single,0.0,95,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,Normal,46338,50745,50745,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.095106,0.095106,0.0

In [33]:
list(filter(lambda x: x.endswith('Age'), df.columns))

['GuarianAge', 'OwnerAge', 'SpouseAge', 'RateIncreaseAge']

In [4]:
duration_scale = lambda x: 2 + np.log(x) if x > 0 else 1
#duration_scale = lambda x: 1 # Rentabilidad solo ajustada por beneficiarios
df = df.query('LapseMid == 0 and LapseFull == 0')
df['Rentabilidad'] = (df.PremiumOutConstant - df.PaidClaims)
df['Rentabilidad_ajustada'] = (df.PremiumOutConstant - df.PaidClaims) * df.Duration.apply(duration_scale)/df.Members
df['Rentabilidad_quintiles'] = 1+pd.qcut(df['Rentabilidad_ajustada'], q=5, labels= False) # Definir bien los quintiles
valor_map = {
    1: 'Sin Valor',
    2: 'Bajo Valor',
    3: 'Medio Valor',
    4: 'Alto Valor',
    5: 'Alto Valor'
}
df['Cliente_valor'] = df['Rentabilidad_quintiles'].map(valor_map)
output_df = df[['tipo_poliza','Rentabilidad', 'Rentabilidad_ajustada', 'Duration', 'Members', 'Rentabilidad_quintiles', 'max_age', 'Cliente_valor']].sort_values('Rentabilidad_ajustada')
output_df

<ipython-input-4-d9c74ce6f961>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rentabilidad'] = (df.PremiumOutConstant - df.PaidClaims)
<ipython-input-4-d9c74ce6f961>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rentabilidad_ajustada'] = (df.PremiumOutConstant - df.PaidClaims) * df.Duration.apply(duration_scale)/df.Members
<ipython-input-4-d9c74ce6f961>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

,tipo_poliza,Rentabilidad,Rentabilidad_ajustada,Duration,Members,Rentabilidad_quintiles,max_age,Cliente_valor
14946,OFF Shore,-764855.510000,-2.900148e+06,6,1,1,58.0,Sin Valor
30426,OFF Shore,-560678.160000,-2.800998e+06,20,1,1,44.0,Sin Valor
55275,ON Shore,-891026.644000,-2.399666e+06,2,1,1,20.0,Sin Valor
54122,ON Shore,-940589.155700,-2.372412e+06,21,2,1,60.0,Sin Valor
44918,OFF Shore,-487755.194400,-1.989769e+06,8,1,1,55.0,Sin Valor
...,...,...,...,...,...,...,...,...
24628,OFF Shore,59290.000000,3.070068e+05,24,1,5,92.0,Alto Valor
2854,OFF Shore,124748.562709,3.368958e+05,30,2,5,89.0,Alto Valor
23398,OFF Shore,150986.000000,3.771428e+05,20,2,5,73.0,Alto Valor
2777,OFF Shore,76109.000000,4.110797e+05,30,1,5,87.0,Alto Valor


In [10]:
np.percentile(df.query('tipo_poliza == "OFF Shore"')['Rentabilidad_ajustada'], q=[0, 20, 40, 60, 80, 100])

array([-2900148.12263375,     7431.43794765,    13461.94531533,
          23360.29703027,    48163.18367251,   461319.09976331])

In [21]:
df.query('tipo_poliza == "OFF Shore"').loc[df['Rentabilidad_ajustada'].between(7431.43794765, 13461.94531533, inclusive='left')]

,PH_ContactBaseID,CoverageFromDate,CoverageThruDate,PolicyYear,Orden,PolicyMonth,BillingFreq,PlanKey,RegionKey,CoverEntity,CoverProduct,CoverPlan,ProductGroup,RiskCountry,RateClassL0,RateClassL1,Duration,Members,FamilyStructure,GuarianAge,OwnerAge,SpouseAge,DependentsCountLT23,DependentsCountAT23,UnknownCount,Unknown1,Unknown2,Unknown3,Unknown4,Unknown5,Unknown6,Unknown7,Unknown8,Unknown9,Renewal,LapseMid,LapseFull,PremiumCalculator,PremiumIn,PremiumOut,PremiumOutConstant,RenewedAT23,flagAlcoholMisuse,flagAsthma,flagAtrialFibrillation,flagCancerLymphoma,flagCancerMetastatic,flagCancerNonMetastatic,flagChronicHeartFailure,flagChronicKidneyDisease,flagChronicPain,flagChronicPulmonaryDisease,flagChronicViralHepatitis,flagCirrhosis,flagDementia,flagDepression,flagDiabetes,flagEpilepsy,flagHypertension,flagHypothyroidism,flagInflammatoryBowel,flagIrritableBowel,flagMultipleSclerosis,flagMyocardialInfarction,flagParkinsons,flagPepticUlcer,flagPeripheralVascular,flagPsoriasis,flagRheumatoidArthritis,flagSchizophrenia,flagSevereConstipation,flagStroke,RateIncrease,RateIncreasePortfolio,RateIncreaseAge,PaidClaims,IncurredClaims,CountClaims,DenialsL12,DeadEndFlag,TransferBupaMexico,TransferBupaEcuador,TransferBupaDR,TransferBupaBolivia,TransferBupaPanama,TransferBupaGuatemala,SuccessorsFoundDiffNumber,tipo_poliza,max_age,Rentabilidad,Rentabilidad_ajustada,Rentabilidad_quintiles,Cliente_valor
3,251806,2020-01-15,2021-01-14,2020,1,1,Semi-Annual,AMP_ET63,AMP_Region 3 Bupa,BIC,Bupa Care Essential,Plan 6,Low,El Salvador,Bupa Care Offshore,Region 3,20,2,Married,0.0,57,57.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,Normal,8158,9232,9232,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.131650,0.131650,0.000000,4199.998,4285.712245,29,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,57.0,5032.002,12569.267396,3,Medio Valor
7,251823,2020-01-15,2021-01-14,2020,1,1,Semi-Annual,AMP_CR25,AMP_Region 5 Bupa,BIC,Bupa Care Critical,Plan 2,Critical,Venezuela,Bupa Care Offshore,Region 5,20,3,Family,0.0,70,52.0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,Critical,4604,4930,4930,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.070808,0.070808,0.000000,0.000,0.000000,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,70.0,4930.000,8209.653370,2,Bajo Valor
25,251973,2015-02-15,2016-02-14,2015,1,2,Semi-Annual,AMP_CR23,AMP_Region 3 Bupa,BIC,Bupa Care Critical,Plan 2,Critical,Bolivia,Bupa Care Offshore,Region 3,15,1,Single,0.0,67,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,Critical,1791,2019,2019,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.127303,0.127303,0.000000,0.000,0.000000,0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0,OFF Shore,67.0,2019.000,9505.553356,2,Bajo Valor
35,252037,2020-03-01,2021-02-28,2020,1,3,Annual,AMP_CR34,AMP_Region 4 Bupa,BIC,Bupa Care Critical,Plan 3,Critical,Jamaica,Bupa Care Offshore,Region 4,20,2,Married,0.0,62,60.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,Critical,3250,3762,3762,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.157538,0.157538,0.000000,0.000,0.000000,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,62.0,3762.000,9396.972407,2,Bajo Valor
37,252058,2014-02-15,2015-02-14,2014,1,2,Semi-Annual,AMP_D2,AMP_Region 2,BIC,Amedex Worldwide,Plan D,Mid,El Salvador,Amedex Worldwide,Region 2,14,2,Single Parent,0.0,37,0.0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,Normal,3581,4322,4322,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.206925,0.206925,0.000000,0.000,0.000000,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,OFF Shore,37.0,4322.000,10025.002889,2,Bajo Valor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59292,1271363,2019-12-01,2020-11-30,2019,1,

In [15]:
df.query('tipo_poliza == "OFF Shore"').groupby('Rentabilidad_quintiles').count()

,PH_ContactBaseID,CoverageFromDate,CoverageThruDate,PolicyYear,Orden,PolicyMonth,BillingFreq,PlanKey,RegionKey,CoverEntity,CoverProduct,CoverPlan,ProductGroup,RiskCountry,RateClassL0,RateClassL1,Duration,Members,FamilyStructure,GuarianAge,OwnerAge,SpouseAge,DependentsCountLT23,DependentsCountAT23,UnknownCount,Unknown1,Unknown2,Unknown3,Unknown4,Unknown5,Unknown6,Unknown7,Unknown8,Unknown9,Renewal,LapseMid,LapseFull,PremiumCalculator,PremiumIn,PremiumOut,PremiumOutConstant,RenewedAT23,flagAlcoholMisuse,flagAsthma,flagAtrialFibrillation,flagCancerLymphoma,flagCancerMetastatic,flagCancerNonMetastatic,flagChronicHeartFailure,flagChronicKidneyDisease,flagChronicPain,flagChronicPulmonaryDisease,flagChronicViralHepatitis,flagCirrhosis,flagDementia,flagDepression,flagDiabetes,flagEpilepsy,flagHypertension,flagHypothyroidism,flagInflammatoryBowel,flagIrritableBowel,flagMultipleSclerosis,flagMyocardialInfarction,flagParkinsons,flagPepticUlcer,flagPeripheralVascular,flagPsoriasis,flagRheumatoidArthritis,flagSchizophrenia,flagSevereConstipation,flagStroke,RateIncrease,RateIncreasePortfolio,RateIncreaseAge,PaidClaims,IncurredClaims,CountClaims,DenialsL12,DeadEndFlag,TransferBupaMexico,TransferBupaEcuador,TransferBupaDR,TransferBupaBolivia,TransferBupaPanama,TransferBupaGuatemala,SuccessorsFoundDiffNumber,tipo_poliza,max_age,Rentabilidad,Rentabilidad_ajustada,Cliente_valor
Rentabilidad_quintiles,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553,2553
2,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205
3,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786,3786
4,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108,4108
5,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537,4537


In [5]:
agg_df = output_df.groupby(['tipo_poliza','Cliente_valor']).agg(
    N_polizas = ('Rentabilidad_quintiles', 'count'),
    Rentabilidad_mean = ('Rentabilidad', 'mean'),
    duracion_mean = ('Duration', 'mean'),
    members_mean = ('Members', 'mean'),
    age_mean = ('max_age', 'mean')
    ).reset_index()
agg_df

,tipo_poliza,Cliente_valor,N_polizas,Rentabilidad_mean,duracion_mean,members_mean,age_mean
0,OFF Shore,Alto Valor,8645,17381.500772,13.261307,1.513129,61.611567
1,OFF Shore,Bajo Valor,3205,5696.943951,9.203432,2.729173,46.182215
2,OFF Shore,Medio Valor,3786,7874.288744,10.689382,2.320919,45.970417
3,OFF Shore,Sin Valor,2553,-19409.396211,8.327458,2.643557,48.503329
4,ON Shore,Alto Valor,1577,13038.289729,12.131262,1.466075,64.316424
5,ON Shore,Bajo Valor,1906,4939.771878,7.090241,2.280693,44.317943
6,ON Shore,Medio Valor,1325,6819.978979,8.759245,1.895849,47.721509
7,ON Shore,Sin Valor,2559,-5277.283750,3.885502,2.660414,43.252833


In [49]:
agg_df.to_csv('segmentos.csv', index=False)

In [74]:
y1 = df.query('tipo_poliza == "OFF Shore"').groupby('Rentabilidad_quintiles').count().PH_ContactBaseID
y1

Rentabilidad_quintiles
1    2553
2    3205
3    3786
4    4108
5    4537
Name: PH_ContactBaseID, dtype: int64

In [7]:
y2 = df.query('tipo_poliza == "ON Shore"').groupby('Rentabilidad_quintiles').count().PH_ContactBaseID
y2

Rentabilidad_quintiles
1    3397
2     816
3    1319
4    1159
5     676
Name: PH_ContactBaseID, dtype: int64

In [11]:
y3 = df.query('tipo_poliza == "OFF Shore"').groupby('Rentabilidad_quintiles').agg(
    n_pol = ('PH_ContactBaseID', 'count'),
    mean_rent = ('Rentabilidad_ajustada', 'mean'))
y3 = y3.round(2)
y3

,n_pol,mean_rent
Rentabilidad_quintiles,,
1,2553,-60243.19
2,3205,8205.27
3,3786,14376.84
4,4108,27315.63
5,4537,82306.43


In [12]:
y4 = df.query('tipo_poliza == "ON Shore"').groupby('Rentabilidad_quintiles').agg(
    n_pol = ('PH_ContactBaseID', 'count'),
    mean_rent = ('Rentabilidad_ajustada', 'mean'))
y4 = y4.round(2)
y4

,n_pol,mean_rent
Rentabilidad_quintiles,,
1,2559,-16771.81
2,1906,7861.68
3,1325,14065.40
4,1003,26540.30
5,574,64217.12


In [78]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
y1 = y1.values
y2 = y2.values
y3 = y3.mean_rent.to_list() + y4.mean_rent.to_list()

# Add traces
fig.add_trace(
    go.Bar(x=[1, 2, 3, 4, 5], y=y1, name="Off Shore", 
    text=y1, textposition='outside', 
    marker_color='rgb(56, 156, 175)'),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=[6, 7, 8, 9, 10], y=y2, name="On Shore", 
        text=y2, textposition='outside', 
        marker_color='rgb(225, 110, 0)'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=list(range(1,11)), y=y3, name='', mode="lines+text", 
        text=y3, textposition="bottom center", 
        line_width=3, line_color='rgb(29, 85, 120)'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="<b>Rentabilidad (ajustada) en Quintiles</b>",
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(1,11)),
        ticktext=[1,2,3,4,5,1,2,3,4,5]
    )
)

# Set x-axis title
fig.update_xaxes(title_text="Quintil")

# Set y-axes titles
fig.update_yaxes(title_text="No. Polizas", secondary_y=False, range=[0, max(y1)*1.1])
fig.update_yaxes(title_text="Rentabilidad promedio", secondary_y=True, range=[min(y3)*1.25, max(y3)*1.5])

fig.show()